In [1]:
from qtools import Submitter
import pandas as pd
import numpy as np
import os
import glob

In [2]:
barcodes_file = '/home/bay001/projects/kris_apobec_20200121/permanent_data2/04_scRNA_RBFOX2_TIA/inputs/RBFOX2_TIA1_barcodes_for_bedgraphs.csv'
tmp_dir = '/home/bay001/projects/kris_apobec_20200121/permanent_data2/04_scRNA_RBFOX2_TIA/tmp'
bed_dir = '/home/bay001/projects/kris_apobec_20200121/permanent_data2/04_scRNA_RBFOX2_TIA/sailor_outputs_individual_barcodes_merged_bedfiles/'
bedgraph_dir = '/home/bay001/projects/kris_apobec_20200121/permanent_data2/04_scRNA_RBFOX2_TIA/sailor_outputs_individual_barcodes_merged_bedgraphs'
output_dir = '/home/bay001/projects/kris_apobec_20200121/permanent_data2/04_scRNA_RBFOX2_TIA/outputs'

In [3]:
conf = 0.0
slop = 25

In [4]:
barcodes = pd.read_csv(barcodes_file)
barcodes

,index,feature,RBFOX2_ORF,TIA1_ORF,lenti_common,new_clusters
0,ACGGTTAAGTGGTTAA-1,RBFOX2_ORF,10.000000,-0.119239,6.722166,APOBEC_STAMP
1,AGGGTTTAGGTCCGAA-1,RBFOX2_ORF,4.260560,-0.011072,4.827667,RBFOX2_STAMP
2,TACCTGCTCCGCTGTT-1,RBFOX2_ORF,7.797372,-0.234867,4.344099,RBFOX2_STAMP
3,CCTTGTGCACCCAACG-1,RBFOX2_ORF,9.327105,-0.355972,4.094241,RBFOX2_STAMP
4,ACAACCATCCTACCGT-1,RBFOX2_ORF,10.000000,-0.290195,3.276557,RBFOX2_STAMP
5,CCTCACAGTGATAGTA-1,TIA1_ORF,-0.197577,10.000000,3.933394,TIA1_STAMP
6,CGTTGGGGTTACAGCT-1,TIA1_ORF,-0.135021,8.158115,3.806437,TIA1_STAMP
7,CCTCATGTCCCGTTGT-1,TIA1_ORF,-0.191547,9.609408,3.410958,TIA1_STAMP
8,TTGGGCGCACTAACCA-1,TIA1_ORF,-0.243492,7.202441,3.311365,TIA1_STAMP
9,AAGCCATAGCTTAAGA-1,TIA1_ORF,-0.203183,9.375294,3.261662,APOBEC_STAMP


In [5]:
def get_sailor_bed_from_barcode_index(row, input_dir=bed_dir):
    barcode = row['index']
    bed = os.path.join(input_dir, 'possorted_genome_bam_MD-{}.bed'.format(barcode))
    assert os.path.exists(bed)
    return bed

In [6]:
barcodes['bed'] = barcodes.apply(get_sailor_bed_from_barcode_index, axis=1)
barcodes

,index,feature,RBFOX2_ORF,TIA1_ORF,lenti_common,new_clusters,bed
0,ACGGTTAAGTGGTTAA-1,RBFOX2_ORF,10.000000,-0.119239,6.722166,APOBEC_STAMP,/home/bay001/projects/kris_apobec_20200121/per...
1,AGGGTTTAGGTCCGAA-1,RBFOX2_ORF,4.260560,-0.011072,4.827667,RBFOX2_STAMP,/home/bay001/projects/kris_apobec_20200121/per...
2,TACCTGCTCCGCTGTT-1,RBFOX2_ORF,7.797372,-0.234867,4.344099,RBFOX2_STAMP,/home/bay001/projects/kris_apobec_20200121/per...
3,CCTTGTGCACCCAACG-1,RBFOX2_ORF,9.327105,-0.355972,4.094241,RBFOX2_STAMP,/home/bay001/projects/kris_apobec_20200121/per...
4,ACAACCATCCTACCGT-1,RBFOX2_ORF,10.000000,-0.290195,3.276557,RBFOX2_STAMP,/home/bay001/projects/kris_apobec_20200121/per...
5,CCTCACAGTGATAGTA-1,TIA1_ORF,-0.197577,10.000000,3.933394,TIA1_STAMP,/home/bay001/projects/kris_apobec_20200121/per...
6,CGTTGGGGTTACAGCT-1,TIA1_ORF,-0.135021,8.158115,3.806437,TIA1_STAMP,/home/bay001/projects/kris_apobec_20200121/per...
7,CCTCATGTCCCGTTGT-1,TIA1_ORF,-0.191547,9.609408,3.410958,TIA1_STAMP,/home/bay001/projects/kris_apobec_20200121/per...
8,TTGGGCGCACTAACCA-1,TIA1_ORF,-0.243492,7.202441,3.311365,TIA1_STAMP,/home/bay001/projects/kris_apobec_20200121/per...
9,AAGCCATAGCTTAAGA-1,TIA1_ORF,-0.203183,9.375294,3.261662,APOBEC_STAMP,/home/bay001/projects/kris_apobec_20200121/per...


In [7]:
def calculate_edit_fraction(row):
    edited, total = row['score'].split(',')
    return int(edited)/float(total)

def bed_to_bedgraph(bed, bedgraph_output_file, bed_output_file, conf, slop):
    df = pd.read_csv(bed, sep='\t', names=['chrom','start','end','name','score','strand'])
    df['chromfx'] = 'chr' + df['chrom'].astype(str)
    df['startfx'] = df['start'] - slop
    df['endfx'] = df['end'] + slop
    df['editfrac'] = df.apply(calculate_edit_fraction, axis=1)
    df = df[df['name'] >= conf]
    df[['chromfx','startfx','endfx','name','editfrac','strand']].to_csv(bed_output_file, sep='\t', index=False, header=False)
    df[['chromfx','startfx','endfx','editfrac']].to_csv(bedgraph_output_file, sep='\t', index=False, header=False)

In [8]:
for row in barcodes.iterrows():
    window_size = slop*2 + 1
    output_file_prefix = os.path.join(bedgraph_dir, '{}_{}_{}_{}_{}_{}.conf{}.windows_{}'.format(
        row[1]['index'], row[1]['feature'], row[1]['RBFOX2_ORF'], row[1]['TIA1_ORF'], row[1]['lenti_common'], row[1]['new_clusters'], conf, window_size
    ))
    bedgraph_output_file = output_file_prefix + '.bedgraph'
    bed_output_file = output_file_prefix + '.bed'
    
    bed_to_bedgraph(row[1]['bed'], bedgraph_output_file, bed_output_file, conf, slop)